<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Github - Get profiles from teams
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Github/Github_Create_issue.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

**Tags:** #github #productivity #code

## Input

### Import libraries

In [1]:
import pandas as pd
import requests
import naas

### Variables

In [2]:
TEAM_URL = "https://api.github.com/orgs/jupyter-naas/teams"
github_token = naas.secret.get("github_token")

## Model

In [3]:
query_url = TEAM_URL
params = {
    "state": "open",
}
headers = {'Authorization': f'token {github_token}'}

In [4]:
main_data = requests.get(query_url, headers=headers, params=params).json()
members_details, GITHUB = [],'jupyter-naas'
for team_info in main_data:
    members_details.append((team_info['name'], team_info['slug'], team_info['description'], team_info['members_url'].strip("{/member}")))

In [5]:
def retrieve_members(team, slug, description, page_link, page_number=1):
    try:
        members = requests.get(page_link, headers=headers, params=params).json()
    except:
        return teams, slugs, descriptions, member_profiles
    
    for member in members:
        member_profiles.append(member['url'])
        teams.append(team)
        slugs.append(slug)
        descriptions.append(description)
    
    if len(members)==30:  ## default members per page shown are 30 ##
        page_number+=1
        if page_number==2:
            page_link+= f'?page={page_number}'
        else:
            index = page_link.find('?page=')
            page_link = page_link[:index+6] + str(page_number)
        return retrieve_members(team, slug, description, page_link, page_number)
    else:
        return teams, slugs, descriptions, member_profiles

member_profiles, teams, slugs, descriptions, page_number=[],[],[],[],1

for info in members_details:
    teams, slugs, descriptions, member_profiles = retrieve_members(info[0], info[1], info[2], info[3], page_number)

    
data = pd.DataFrame(columns=['TEAM', 'SLUG','TEAM_DESCRIPTION', 'member_profile'])
data['TEAM'], data['SLUG'], data['TEAM_DESCRIPTION'], data['member_profile'] = teams, slugs, descriptions, member_profiles
data['GITHUB'] = 'jupyter-naas'

print(data.shape)

(61, 5)


In [6]:
for idx, profile in enumerate(data['member_profile']):
    details = requests.get(profile, headers=headers, params= params).json()
    data.loc[idx,'NAME'], data.loc[idx,'EMAIL'], data.loc[idx,'LOCATION'] = details['name'], details['email'], details['location']
    data.loc[idx,'ORGANIZATION'], data.loc[idx,'BIO'], data.loc[idx,'LOGIN_NAME'] = details['company'], details['bio'], details['login']
    data.loc[idx,'TWITTER'], data.loc[idx,'CREATED_AT'], data.loc[idx,'UPDATED_AT'] = details['twitter_username'], details['created_at'], details['updated_at']

## Output

In [7]:
print(f'Dataset size -> {data.shape}')
data

Dataset size -> (61, 14)


,TEAM,SLUG,TEAM_DESCRIPTION,member_profile,GITHUB,NAME,EMAIL,LOCATION,ORGANIZATION,BIO,LOGIN_NAME,TWITTER,CREATED_AT,UPDATED_AT
0,core-team,core-team,Naas core team group,https://api.github.com/users/riderx,jupyter-naas,Martin DONADIEU,martindonadieu@gmail.com,Remote,@Forgr-ee @Captime @INDIEMAKERS,🧞‍♂️INDIE MAKER | 🎙Podcaster | 🏄‍♀️\r\nLife Go...,riderx,martindonadieu,2013-04-07T13:49:12Z,2022-01-19T02:46:44Z
1,core-team,core-team,Naas core team group,https://api.github.com/users/navinkarkera,jupyter-naas,Navin Karkera,navin@disroot.org,None,None,None,navinkarkera,None,2015-02-07T08:20:41Z,2021-06-17T07:22:37Z
2,core-team,core-team,Naas core team group,https://api.github.com/users/jravenel,jupyter-naas,Jeremy RAVENEL,jeremy.ravenel@cashstory.com,Everywhere,@getcashstory,Founder @CashStory | Opensourcing @jupyter-naa...,jravenel,ravenel_jeremy,2016-08-16T05:43:44Z,2022-01-02T17:13:30Z
3,core-team,core-team,Naas core team group,https://api.github.com/users/Dr0p42,jupyter-naas,Maxime Jublou,maxime@jublou.fr,Pyrénées,naas.ai,PGP Key: https://keybase.io/jubloum,Dr0p42,None,2016-09-22T08:07:56Z,2022-01-24T16:20:38Z
4,core-team,core-team,Naas core team group,https://api.github.com/users/imasdekar,jupyter-naas,Ishan Masdekar,imasdekar@disroot.org,None,None,None,imasdekar,None,2017-06-18T15:55:41Z,2021-12-28T13:26:19Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,opensource-contributors,opensource-contributors,Naas opensource contributors group,https://api.github.com/users/OketunjiOludolapo,jupyter-naas,Oketunji Oludolapo,None,None,None,I'm a Data Scientist and i love sharing ideas ...,OketunjiOludolapo,None,2021-06-14T21:40:48Z,2022-01-14T01:10:53Z
57,opensource-contributors,opensource-contributors,Naas opensource contributors group,https://api.github.com/users/krishkatyal,jupyter-naas,Krish Katyal,ka1786@srmist.edu.in,"Gurgaon, Haryana, India",None,Associate @NextTechLab | ML Engineer @td-locus...,krishkatyal,None,2021-08-26T15:29:11Z,2022-01-26T10:25:54Z
58,opensource-contributors,opensource-contributors,Naas opensource contributors group,https://api.github.com/users/Guru0593,jupyter-naas,None,None,None,None,None,Guru0593,None,2021-08-30T15:36:13Z,2021-12-28T12:23:57Z
59,opensource-contributors,opensource-contributors,Naas opensource contributors group,https://api.github.com/users/RiddhiDeshpande222,jupyter-naas,Riddhi Deshpande,None,None,None,Data Analytics Enthusiast | Tableau | PowerBI,RiddhiDeshpande222,None,2021-11-27T08:34:40Z,2022-01-22T19:12:52Z
